In [1]:
import os
import sys
import datetime

import pickle
import numpy as np
from sklearn.model_selection import train_test_split

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../../")))
from SRModels.loading_methods import load_dataset_as_patches
from SRModels.deep_learning_models.SRCNN_model import SRCNNModel
from SRModels.constants import SRCNN_PATCH_SIZE, SRCNN_STRIDE, RANDOM_SEED

In [2]:
HR_ROOT = os.path.abspath(os.path.join(os.getcwd(), "../../data/images/HR"))
LR_ROOT = os.path.abspath(os.path.join(os.getcwd(), "../../data/images/LR"))
INTERP_MAP_PATH = os.path.abspath(os.path.join(os.getcwd(), "../../data/images/interpolation_map.pkl"))

In [3]:
# X -> Low-resolution patches (model input) (Low-resolution images with same size as Y but noised)
# Y -> High-resolution patches (target)
X, Y, hr_h, hr_w = load_dataset_as_patches(HR_ROOT, LR_ROOT, mode="srcnn", patch_size=SRCNN_PATCH_SIZE, stride=SRCNN_STRIDE, interpolation_map_path=INTERP_MAP_PATH)

# Reduce dataset size with percentage
X = X[:int(0.7 * len(X))]
Y = Y[:int(0.7 * len(Y))]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, shuffle=True, random_state=RANDOM_SEED)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.1, shuffle=True, random_state=RANDOM_SEED)

print(f"X shape: {X.shape}, Y shape: {Y.shape}")
print(f"X_train shape: {X_train.shape}, Y_train shape: {Y_train.shape}")
print(f"X_val shape: {X_val.shape}, Y_val shape: {Y_val.shape}")
print(f"X_test shape: {X_test.shape}, Y_test shape: {Y_test.shape}")

X shape: (333251, 24, 24, 3), Y shape: (333251, 24, 24, 3)
X_train shape: (269932, 24, 24, 3), Y_train shape: (269932, 24, 24, 3)
X_val shape: (29993, 24, 24, 3), Y_val shape: (29993, 24, 24, 3)
X_test shape: (33326, 24, 24, 3), Y_test shape: (33326, 24, 24, 3)


In [4]:
model = SRCNNModel()

model.setup_model(input_shape=X_train.shape[1:], learning_rate=1e-4)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 24, 96)        23424     
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        3104      
                                                                 
 conv2d_2 (Conv2D)           (None, 24, 24, 3)         2403      
                                                                 
Total params: 28,931
Trainable params: 28,931
Non-trainable params: 0
_________________________________________________________________


In [5]:
# Train SRCNN and capture callbacks for metrics
time_cb, mem_cb = model.fit(
    X_train, Y_train, X_val, Y_val,
    batch_size=32,
    epochs=100
)

Training on GPU: /physical_device:GPU:0
Epoch 1/100
8436/8436 [==============================] - 35s 4ms/step - loss: 0.0321 - psnr: 29.1668 - ssim: 0.8019 - val_loss: 0.0282 - val_psnr: 30.1107 - val_ssim: 0.8142 - lr: 1.0000e-04 - epoch_time_sec: 34.8530 - gpu_mean_current_mb: 3756.6389 - gpu_peak_mb: 3999.7629
Epoch 2/100
8436/8436 [==============================] - 36s 4ms/step - loss: 0.0283 - psnr: 30.0607 - ssim: 0.8136 - val_loss: 0.0279 - val_psnr: 30.1962 - val_ssim: 0.8160 - lr: 1.0000e-04 - epoch_time_sec: 35.8175 - gpu_mean_current_mb: 3954.4885 - gpu_peak_mb: 3999.7629
Epoch 3/100
8436/8436 [==============================] - 35s 4ms/step - loss: 0.0281 - psnr: 30.1509 - ssim: 0.8151 - val_loss: 0.0279 - val_psnr: 30.1742 - val_ssim: 0.8164 - lr: 1.0000e-04 - epoch_time_sec: 35.0484 - gpu_mean_current_mb: 3954.4885 - gpu_peak_mb: 3999.7629
Epoch 4/100
8436/8436 [==============================] - 47s 6ms/step - loss: 0.0279 - psnr: 30.1978 - ssim: 0.8161 - val_loss: 0.0278 

In [6]:
# Evaluate and prepare metrics dictionary
results = model.evaluate(X_test, Y_test)
metrics_dict = {
    "eval_loss": float(results[0]),
    "eval_psnr": float(results[1]),
    "eval_ssim": float(results[2]),
    "epoch_time_sec": time_cb.mean_time_value(),
    "memory": mem_cb.as_dict()
}
print(metrics_dict)

1042/1042 [==============================] - 4s 3ms/step - loss: 0.0272 - psnr: 30.4773 - ssim: 0.8213
Loss: 0.0272, PSNR: 30.48 dB, SSIM: 0.8213
{'eval_loss': 0.027160510420799255, 'eval_psnr': 30.47734832763672, 'eval_ssim': 0.821340024471283, 'epoch_time_sec': 37.743002903030295, 'memory': {'gpu_mean_current_mb': 3948.494558623343, 'gpu_peak_mb': 3999.762939453125}}


In [7]:
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

# Save the trained model
model.save(directory=f"models/SRCNN/SRCNN_{timestamp}", timestamp=timestamp)

# Save hr_h and hr_w for later use in the reconstruction of the images
with open(os.path.abspath(os.path.join(os.getcwd(), f"models/SRCNN/SRCNN_{timestamp}/SRCNN_{timestamp}_hrh_hrw.pkl")), "wb") as f:
    pickle.dump((hr_h, hr_w), f)

Model saved to models/SRCNN/SRCNN_20250909_160258\SRCNN_20250909_160258.h5


In [8]:
# Save evaluation/time/memory metrics next to the model
metrics_path = os.path.abspath(os.path.join(os.getcwd(), f"models/SRCNN/SRCNN_{timestamp}/SRCNN_{timestamp}_metrics.pkl"))

with open(metrics_path, "wb") as f:
    pickle.dump(metrics_dict, f)
    
print(f"Saved metrics to {metrics_path}")

Saved metrics to c:\Users\bgmanuel\InteligenciaArtificial\MasterInteligenciaArtificial\Periodo2\TFM\Super-Resolution-Images-for-3D-Printing-Defect-Detection\SRModels\deep_learning_models\models\SRCNN\SRCNN_20250909_160258\SRCNN_20250909_160258_metrics.pkl
